In [ ]:
import pandas as pd
import spacy

df_train= pd.read_csv("politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [ ]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_test.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [ ]:
df_train.drop('label', axis=1, inplace=True)
df_test.drop('label', axis=1, inplace=True)

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('es_core_news_lg')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [ ]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df_train= pd.read_csv('/content/drive/My Drive/TFM/train_tokenized.csv')
df_test= pd.read_csv('/content/drive/My Drive/TFM/test_tokenized.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_train['ideology_multiclass'] = label_encoder.fit_transform(df_train['ideology_multiclass'])
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

vectorizador = TfidfVectorizer()

X_train_tfidf = vectorizador.fit_transform(df_train['clean_tweet'])
X_test_tfidf = vectorizador.transform(df_test['clean_tweet'])

y_train = df_train['ideology_multiclass']
y_test = df_test['ideology_multiclass']

# Definir el modelo y los parámetros para GridSearchCV
model_svm = SVC(random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'kernel': ['rbf', 'linear', 'poly']
}

# Crear un GridSearchCV
clf = GridSearchCV(model_svm, param_grid, scoring='f1_macro', cv=5, verbose=1, n_jobs=-1)

# Ajustar el modelo
clf.fit(X_train_tfidf, y_train)

# Mejores parámetros y mejor score
print("Mejores parámetros:", clf.best_params_)
print("Mejor puntuación de cross-validation (macro F1):", clf.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Mejores parámetros: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Mejor puntuación de cross-validation (macro F1): 0.9307337262749676


In [6]:
y_pred = clf.predict(X_test_tfidf)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print(macro_f1)
print(classification_report(y_test, y_pred))

0.6514003134117399
              precision    recall  f1-score   support

           0       0.76      0.56      0.64       117
           1       0.62      0.84      0.72       210
           2       0.70      0.67      0.68       153
           3       0.93      0.40      0.56        67

    accuracy                           0.68       547
   macro avg       0.75      0.62      0.65       547
weighted avg       0.71      0.68      0.67       547

